In [130]:
import random
import datetime
import numpy as np
from threading import Thread

In [153]:
# Settings
MAX_POINT_COORDINATE_VALUE = 40000
NUMBER_OF_RANDOM_POINTS_PER_SWEEP = 1000
NUMBER_OF_RANDOM_POINTS_PER_SWEEP_MULTI = 100000
EPS = 2
NUMBER_OF_MODELS = 30
NUMBER_OF_THREADS = 10

In [132]:
model = {
    "dimensions" : 4,
    "equation" : "4*x1 + 6*x2 + 3*x3 + 12*x4",
    "optimization" : "max",
    "conditions" : [
        "x1 + 2*x2 + 1.5*x3 + 6*x4 <= 90000",
        "2*x1 + 2*x2 + 1.5*x3 + 4*x4 <= 120000",
        "x1 >= 0",
        "x2 >= 0",
        "x3 >= 0",
        "x4 >= 0"
    ]
}

In [133]:
supported_operations = {
    "log" : "np.log",
    "sin" : "np.sin",
    "cos" : "np.cos",
    "tan" : "np.tan",
    "sqrt" : "np.sqrt",
    "absolute" : "np.absolute"
}

In [134]:
# Using string replace substitute variables with selected values
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [135]:
# Initial processing of task conditions
def model_preprocessing(model, supported_operations):
    preprocessed_model = model.copy()
    preprocessed_model["conditions"] = [replace_all(condition, supported_operations) for condition in model["conditions"]]
    preprocessed_model["equation"] = replace_all(model["equation"], supported_operations)
    return preprocessed_model

In [136]:
# Evaluate single condition
def evaluate_condition(condition, point_position):
    condition = replace_all(condition, point_position)
    return eval(condition)

In [137]:
# Validate conditions for point
def validate_single_point(conditions, point_position):
    return all([evaluate_condition(condition, point_position) for condition in conditions])

In [138]:
# Generate dictionary with start point position
def generate_start_point(dimensions, value):
    return {"x"+str(i): str(value) for i in range(1, dimensions+1)}

In [139]:
# Generate dictionary with random point position
def generate_point(dimensions):
    return {"x"+str(i): str(random.randint(0,MAX_POINT_COORDINATE_VALUE)) for i in range(1, dimensions+1)}

In [140]:
# Generate dictionary with random point position in the vicinity of other point
def generate_point_in_vicinity(point, r):
    return {key: str(int(value) + random_number_in_vicinity(r)) for key, value in point.items()}

In [141]:
# Generate number in specific vicinity from 0
def random_number_in_vicinity(r):
    return int(random.randint(0,2*r) - r)

In [142]:
# Perform single sweep in solution optimization

def single_sweep(model, num_of_points, center, r):
    best_point = None
    best_score = None
    
    if validate_single_point(model["conditions"], center):
        best_point = center
        equation = replace_all(model["equation"], best_point)
        best_score = eval(equation)
    
    for i in range(num_of_points):
        # Generate random point position
        random_point = generate_point_in_vicinity(center, r)
            
        if validate_single_point(model["conditions"], random_point):
            equation = replace_all(model["equation"], random_point)
            value = eval(equation)
            if model["optimization"] is "max" and (best_score == None or best_score < value):
                best_point = random_point
                best_score = value
            elif model["optimization"] is "min" and (best_score == None or best_score > value):
                best_point = random_point
                best_score = value
        
    return best_point, best_score

In [143]:
# Perform single sweep in solution optimization for single thread

def single_sweep_in_thread(model, num_of_points, center, r, scores, points, index):
    best_point = None
    best_score = None
    
    if validate_single_point(model["conditions"], center):
        best_point = center
        equation = replace_all(model["equation"], best_point)
        best_score = eval(equation)
    
    for i in range(num_of_points):
        # Generate random point position
        random_point = generate_point_in_vicinity(center, r)
            
        if validate_single_point(model["conditions"], random_point):
            equation = replace_all(model["equation"], random_point)
            value = eval(equation)
            if model["optimization"] is "max" and (best_score == None or best_score < value):
                best_point = random_point
                best_score = value
            elif model["optimization"] is "min" and (best_score == None or best_score > value):
                best_point = random_point
                best_score = value
                
    points[index] = best_point
    scores[index] = best_score
    return

In [144]:
def optimize_solution(model, min_r, max_no_improvement, verbose = True):
    r = MAX_POINT_COORDINATE_VALUE
    center = generate_start_point(model["dimensions"], int(MAX_POINT_COORDINATE_VALUE/2))
    prev_score = 100000000
    steps_with_no_improvement = 0
    
    if verbose:
        ts = datetime.datetime.now()
        print("Start time: " + str(ts))
        t_per_sweep = ts

    while r > min_r:
        point, score = single_sweep(model, NUMBER_OF_RANDOM_POINTS_PER_SWEEP, center, r)
        
        if verbose:
            current_time = datetime.datetime.now()
            time_of_sweep = current_time - t_per_sweep
            t_per_sweep = current_time
            print("Time_of_sweep: " + str(time_of_sweep))
            print("Score: " + str(score))
            print("R: " + str(r))
            print("Point: " + str(point))
            print("---------------------------------")
        
        if score == None:
            steps_with_no_improvement += 1
            
        elif prev_score == score:
            steps_with_no_improvement += 1
            
            if steps_with_no_improvement >= max_no_improvement:
                
                if verbose:
                    tf = datetime.datetime.now()
                    te = tf - ts
                    print("End time: " + str(tf))
                    print("Run time: " + str(te))

                return point, score
            
            else:
                prev_score = score
                center = point
                r = int(r/2)
                
        elif abs(prev_score - score) <= EPS:
            
            if verbose:
                tf = datetime.datetime.now()
                te = tf - ts
                print("End time: " + str(tf))
                print("Run time: " + str(te))
            
            return point, score
        else:
            steps_with_no_improvement = 0
            prev_score = score
            center = point
            r = int(r/2)
            
    return point, score

In [154]:
def multi_thread_optimize_solution(model):
    r = MAX_POINT_COORDINATE_VALUE
    center = generate_start_point(model["dimensions"], int(MAX_POINT_COORDINATE_VALUE/2))
    prev_score = 100000000
    
    ts = datetime.datetime.now()
    print("Start time: " + str(ts))
    
    t_per_sweep = ts
    
    while True:
        iterations_per_thread = int(NUMBER_OF_RANDOM_POINTS_PER_SWEEP_MULTI/NUMBER_OF_THREADS)
        
        threads = [None] * NUMBER_OF_THREADS
        scores = [None] * NUMBER_OF_THREADS
        points = [None] * NUMBER_OF_THREADS
        
        for i in range(NUMBER_OF_THREADS):
            threads[i] = Thread(target=single_sweep_in_thread, 
                                args=(model, iterations_per_thread, center, r, scores, points, i))
            threads[i].start()
            
        for i in range(len(threads)):
            threads[i].join()
            
        print(scores)
        
        if model["optimization"] is "max":
            optimal_index = np.argmax(np.asarray(scores))
        elif model["optimization"] is "min":
            optimal_index = np.argmin(np.asarray(scores))
            
        point = points[optimal_index]
        score = scores[optimal_index]
        
        current_time = datetime.datetime.now()
        time_of_sweep = current_time - t_per_sweep
        t_per_sweep = current_time
        
        print("Time_of_sweep: " + str(time_of_sweep))
        print("Score: " + str(score))
        print("R: " + str(r))
        print("Point: " + str(point))
        print("---------------------------------")
        
        if abs(prev_score - score) <= EPS:
            
            tf = datetime.datetime.now()
            te = tf - ts
            print("End time: " + str(tf))
            print("Run time: " + str(te))
            
            return point, score
        else:
            prev_score = score
            center = point
            r = int(r/2)

In [150]:
# Test script

processed_model = model_preprocessing(model, supported_operations)

points = [None] * NUMBER_OF_MODELS
scores = [None] * NUMBER_OF_MODELS

for i in range(NUMBER_OF_MODELS):
    points[i], scores[i] = optimize_solution(processed_model, 1, 5, verbose=False)
    print("Iteration {};  Score: {}".format(i, scores[i])) 

if model["optimization"] is "max":
    optimal_index = np.argmax(np.asarray(scores))
elif model["optimization"] is "min":
    optimal_index = np.argmin(np.asarray(scores))

point = points[optimal_index]
score = scores[optimal_index]
    
print("Score: " + str(score))
print("Point: " + str(point))

Iteration 0;  Score: 294118
Iteration 1;  Score: 290274
Iteration 2;  Score: 290552
Iteration 3;  Score: 295284
Iteration 4;  Score: 284938
Iteration 5;  Score: 299076
Iteration 6;  Score: 271409
Iteration 7;  Score: 295966
Iteration 8;  Score: 289812
Iteration 9;  Score: 296994
Iteration 10;  Score: 297308
Iteration 11;  Score: 294820
Iteration 12;  Score: 263274
Iteration 13;  Score: 294178
Iteration 14;  Score: 296508
Iteration 15;  Score: 279422
Iteration 16;  Score: 290704
Iteration 17;  Score: 288054
Iteration 18;  Score: 299843
Iteration 19;  Score: 295311
Iteration 20;  Score: 296144
Iteration 21;  Score: 297787
Iteration 22;  Score: 299752
Iteration 23;  Score: 296308
Iteration 24;  Score: 293390
Iteration 25;  Score: 298792
Iteration 26;  Score: 296122
Iteration 27;  Score: 298490
Iteration 28;  Score: 299648
Iteration 29;  Score: 298616
[294118, 290274, 290552, 295284, 284938, 299076, 271409, 295966, 289812, 296994, 297308, 294820, 263274, 294178, 296508, 279422, 290704, 288

In [155]:
# Test script multithread

processed_model = model_preprocessing(model, supported_operations)
point, score = multi_thread_optimize_solution(processed_model)

Start time: 2018-04-22 20:03:44.934438
[264136, 263533, 279164, 272272, 255504, 254444, 272712, 278950, 287187, 277455]
Time_of_sweep: 0:00:06.957298
Score: 287187
R: 40000
Point: {'x1': '22737', 'x2': '30910', 'x3': '3225', 'x4': '92'}
---------------------------------
[287187, 287187, 287187, 287187, 287187, 287187, 287187, 288466, 287187, 294585]
Time_of_sweep: 0:00:06.986156
Score: 294585
R: 20000
Point: {'x1': '30930', 'x2': '27536', 'x3': '1235', 'x4': '162'}
---------------------------------
[294585, 294585, 294585, 294585, 294886, 294585, 294585, 294585, 294585, 294585]
Time_of_sweep: 0:00:06.901872
Score: 294886
R: 10000
Point: {'x1': '31132', 'x2': '26729', 'x3': '660', 'x4': '667'}
---------------------------------
[294886, 294886, 295136, 298317, 296104, 297400, 294886, 296381, 294886, 296010]
Time_of_sweep: 0:00:06.972663
Score: 298317
R: 5000
Point: {'x1': '30309', 'x2': '28956', 'x3': '87', 'x4': '257'}
---------------------------------
[299009, 298317, 298546, 298317, 2